# Homework06

Exercises to practice pandas, data analysis and regression

## Goals

- Understand the effects of pre-processing data
- Get familiar with the ML flow: encode -> normalize -> train -> evaluate
- Understand the difference between regression and classification tasks
- Build an intuition for different regression models

### Setup

Run the following 2 cells to import all necessary libraries and helpers for this homework.

In [ ]:
!wget -q https://github.com/PSAM-5020-2025S-A/5020-utils/raw/main/src/data_utils.py

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.preprocessing import OrdinalEncoder

from data_utils import object_from_json_url
from data_utils import StandardScaler
from data_utils import LinearRegression, SGDRegressor
from data_utils import PolynomialFeatures
from data_utils import regression_error

### Load Dataset

Let's load up the full [ANSUR](https://www.openlab.psu.edu/ansur2/) dataset that we looked at briefly in [Week 02](https://github.com/DM-GY-9103-2024F-H/WK02).

This is the dataset that has anthropometric information about U.S. Army personnel.

In [ ]:
# Load Dataset
ANSUR_FILE = "https://raw.githubusercontent.com/PSAM-5020-2025S-A/5020-utils/main/datasets/json/ansur.json"
ansur_data = object_from_json_url(ANSUR_FILE)

# Look at first 2 records
ansur_data[:2]

#### Nested data

This is that *nested* dataset from Week 02.

# 🤔

Let's load it into a `DataFrame` to see what happens.

In [ ]:
# Read into DataFrame
ansur_df = pd.DataFrame.from_records(ansur_data)
ansur_df.head()


# 😓🙄

That didn't work too well. We ended up with objects in our columns.

Luckily, our `DataFrame` library has a function called [`json_normalize()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.json_normalize.html) that can help.

In [ ]:
# Read into DataFrame
ansur_df = pd.json_normalize(ansur_data)
ansur_df.head()

Much better. `DataFrames` are magic.

#### Data Exploration

Before we start creating models, let's do a little bit of data analysis and get a feeling for the shapes, distributions and relationships of our data.

1. Print `min`, `max` and `average` values for all of the features.
2. Print `covariance` tables for `age`, `ear.length` and `head.circumference`.
3. Plot `age`, `ear.length` and `head.circumference` versus the $1$ *feature* that is most correlated to each of them.

Don't forget to *encode* and *normalize* the data.

In [ ]:
# Work on Data Exploration here

### Encode non-numerical features
gender = ['F', 'M']
gender_encoder = OrdinalEncoder(categories=[gender])
ccc_vals = gender_encoder.fit_transform(ansur_df[["gender"]].values)
ansur_df[["gender"]] = ccc_vals

## 1. Print min, max, avg
for feat in ansur_df.columns:
  print(feat)
  print("\tmin:", ansur_df[feat].min())
  print("\tmax:", ansur_df[feat].max())
  print("\tavg:", ansur_df[feat].mean())

### Normalize all data
ansur_scaler = StandardScaler()
ansur_scaled = ansur_scaler.fit_transform(ansur_df)

## 2. Print Covariances
print(ansur_df[["age", "ear.length", "head.circumference"]].cov())

## 3. Plot features most correlated to age, ear length and head circumference
for feat in ["age", "ear.length", "head.circumference"]:
  plt.plot(ansur_df[feat], ansur_df["span"], marker="o", linestyle="", alpha=0.3)
  plt.xlabel(feat)
  plt.ylabel("span")
  plt.show()

### Interpretation

<span style="color:hotpink;">
Does anything stand out about these graphs? Or the correlations?<br>
Are correlations symmetric? Does the feature most correlated to ear length also have ear length as its most correlated pair?
</span>

<span style="color:orange;">
ANSWER: The correlation between span and ear.length, and span and head.circumference seem to relate in positive way almost symmetric. 
</span>

### Regression

Now, we want to create a regression model to predict `head.circumference` from the data.

From our [Week 06](https://github.com/PSAM-5020-2025S-A/WK06) notebook, we can create a regression model by following these steps:

1. Load dataset (done! 🎉)
2. Encode label features as numbers (done! ⚡️)
3. Normalize the data (done! 🍾)
4. Separate the outcome variable and the input features
5. Create a regression model using all features
6. Run model on training data and measure error
7. Plot predictions and interpret results
8. Run model on test data, measure error, plot predictions, interpret results

In [ ]:
# Work on Regression Model here

## Separate outcome variable and input features
head = ansur_scaled[["head.circumference"]]
features = ansur_scaled.drop(columns=["head.circumference"])

## Create a regression model
model = LinearRegression()
poly = PolynomialFeatures(degree=5, include_bias=False)
features_poly = poly.fit_transform(features)

model.fit(features_poly, head)
predicted_scaled = model.predict(features_poly)
predicted = ansur_scaler.inverse_transform(predicted_scaled)

## Measure error on training data
regression_error(ansur_df["head.circumference"], predicted["head.circumference"])

## Plot predictions and interpret results
head_original = sorted(ansur_df["head.circumference"])
head_predicted = sorted(predicted["head.circumference"])

plt.plot(head_original, marker='o', linestyle='', alpha=0.3)
plt.plot(head_predicted, color='r', marker='o', markersize='3', linestyle='', alpha=0.1)
plt.ylabel("span")
plt.show()

In [ ]:
## Load Test Data
ANSUR_TEST_FILE = "https://raw.githubusercontent.com/PSAM-5020-2025S-A/5020-utils/main/datasets/json/ansur-test.json"

ansur_test_data = object_from_json_url(ANSUR_TEST_FILE)
ansur_test_df = pd.json_normalize(ansur_test_data)

ansur_test_encoded_df = ansur_test_df.copy()

g_vals = gender_encoder.transform(ansur_test_df[["gender"]].values)
ansur_test_encoded_df[["gender"]] = g_vals

ansur_test_scaled_df = ansur_scaler.transform(ansur_test_encoded_df)

In [ ]:
## Run model on test data
head = ansur_test_scaled_df["head.circumference"]
features = ansur_test_scaled_df.drop(columns=["head.circumference"])
features_poly = poly.fit_transform(features)
model.fit(features_poly, head)
predicted_scaled = model.predict(features_poly)
predicted = ansur_scaler.inverse_transform(predicted_scaled)

## Measure error on test data
regression_error(ansur_test_encoded_df["head.circumference"], predicted["head.circumference"])

## Plot predictions and interpret results
head_test = sorted(ansur_test_encoded_df["head.circumference"])
head_predicted = sorted(predicted["head.circumference"])

plt.plot(head_test, marker='o', linestyle='', alpha=0.3)
plt.plot(head_predicted, color='r', marker='o', markersize='3', linestyle='', alpha=0.1)
plt.ylabel("span")
plt.show()

### Interpretation

<span style="color:hotpink;">
How well does your model perform?<br>
How could you improve it?<br>
Are there ranges of circumferences that don't get predicted well?
</span>

<span style="color:orange;">
ANSWER: The model is perform very well as I can see!<br>
But, maybe too well that I thought it just copied itself. I'm not sure - too good to be true!?<br> However, when I change PolynomialFeatures degree, the plot is changed and the error showed 7.xxx<br> So I hope it's ok👌
</span>